# Scraping MERISTATION

In [5]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

In [2]:
review_url = 'https://as.com/meristation/2020/09/19/analisis/1600499631_945194.html'
review_html = requests.get(review_url).content
review_html[:50]

b'<!DOCTYPE html>\n<html lang="es">\n<head>\n<meta char'

In [3]:
soup = BeautifulSoup(review_html, 'lxml')

In [7]:
p_tags = soup.find('div', {'class': 'art-body'}).find_all('p')

In [8]:
review = [tag.text for tag in p_tags]
review = '-'.join(review)

In [9]:
review

'Mucho ha llovido desde el lanzamiento de Spelunky HD, en 2012 (aquí podéis leer el análisis redactado en su momento por nuestro compañero Salva Fernández). Pero más ha llovido aún desde el lanzamiento del Spelunky original, allá por 2008. Una docena de años después, en septiembre de 2020, nos llega Spelunky 2, título a priori exclusivo en consolas de PlayStation 4, cuya versión está disponible un par de semanas antes que en PC. Antes de nada, queremos deciros que el titular no es, en absoluto, clickbait. Quien escribe estas líneas ha visto y ha jugado mucho, y también ha echado muchas horas a Spelunky en Xbox 360, consiguiendo todos sus logros. Y esta segunda parte coge la fórmula original y la retuerce de tal forma que el resultado es el videojuego más difícil no solo de todo 2020, sino de toda la década. Ahí es nada.-"Más, y, además, mejor". Con esta frase tan escueta podríamos resumir, en líneas generales, lo que nos ha parecido Spelunky 2. Este título coge todo lo que hizo grande 

# Scraping Análisis

In [12]:
review_url = 'https://as.com/meristation/analisis/'
html = requests.get(review_url).content
html[:50]

b'<!DOCTYPE html>\n<html lang="es">\n<head>\n<meta char'

In [13]:
soup = BeautifulSoup(html, 'lxml')
articles = soup.find_all('h2')

In [16]:
#Extraer títulos

titles = []

for a in articles:
    if a.find('a') == None:
        pass
    else:
        titles.append(a.find('a')['title'])
    
titles

['Análisis Bounty Battle; el título de lucha de los héroes indie',
 'eFootball PES 2021, análisis. Rueda el balón, pero poco más',
 'Milky Way Prince: The Vampire Prince, análisis. Amor tóxico',
 'Spinch, análisis PC y Switch: Una experiencia sensorial',
 'Spelunky 2, análisis: el videojuego más hardcore de la década',
 'WRC 9, análisis Xbox One',
 'Inertial Drift, análisis',
 'Super Mario 3D All-Star, análisis',
 'GeForce RTX 3080, Análisis de una tarjeta gráfica de nueva generación',
 '13 Sentinels: Aegis Rim, análisis. Una historia de otro nivel',
 'Vader Immortal: A Star Wars VR Series, análisis. Cuando te falla el sable de luz',
 'Ary and the Secret of Seasons, análisis Switch',
 'MX vs. ATV All Out, análisis. Hay que aprender a pasar página',
 'West of Dead, análisis',
 'NBA 2K21, análisis',
 'Windbound: Brave The Storm, análisis',
 "New Super Lucky's Tale, análisis",
 'Railway Empire Complete Collection, análisis PC',
 'Nexomon Extinction, Análisis. Una alternativa real para Pok

In [17]:
#Extraer links

links = []

for a in articles:
    if a.find('a') == None:
        pass
    else:
        links.append(a.find('a')['href'])
    
links

['https://as.com/meristation/2020/09/22/analisis/1600764887_024298.html',
 'https://as.com/meristation/2020/09/22/analisis/1600760878_562894.html',
 'https://as.com/meristation/2020/09/21/analisis/1600675002_511518.html',
 'https://as.com/meristation/2020/09/20/analisis/1600588999_099915.html',
 'https://as.com/meristation/2020/09/19/analisis/1600499631_945194.html',
 'https://as.com/meristation/2020/09/18/analisis/1600418364_262891.html',
 'https://as.com/meristation/2020/09/18/analisis/1600418263_422657.html',
 'https://as.com/meristation/2020/09/16/analisis/1600261344_550903.html',
 'https://as.com/meristation/2020/09/16/mexico/1600234167_066181.html',
 'https://as.com/meristation/2020/09/15/analisis/1600181818_035725.html',
 'https://as.com/meristation/2020/09/15/analisis/1600166091_128370.html',
 'https://as.com/meristation/2020/09/14/analisis/1600097234_505564.html',
 'https://as.com/meristation/2020/09/13/analisis/1599985057_505947.html',
 'https://as.com/meristation/2020/09/12/

In [30]:
len(reviews_col)

45

### SCRAPING REVIEWS

In [21]:
reviews_col = []

for link in links:
    
    #Request content and wait
    review_url = link
    review_html = requests.get(review_url).content
    time.sleep(3)
    
    #Extract text
    soup = BeautifulSoup(review_html, 'lxml')
    articles = soup.find_all('h2')
    p_tags = soup.find('div', {'class': 'art-body'}).find_all('p')
    
    #Clean text
    review = [tag.text for tag in p_tags]
    review = '-'.join(review)
    
    #Add to a list
    reviews_col.append(review)

### Create DataFrame

In [33]:
meristation = pd.DataFrame({'title': titles, 
                           'author': np.nan, 
                           'review': reviews_col, 
                           'url': links 
                           })

meristation.head()

,title,author,review,url
0,Análisis Bounty Battle; el título de lucha de ...,NaN,Después del anuncio de su retraso indefinido e...,https://as.com/meristation/2020/09/22/analisis...
1,"eFootball PES 2021, análisis. Rueda el balón, ...",NaN,La temporada futbolística acabó de manera atíp...,https://as.com/meristation/2020/09/22/analisis...
2,"Milky Way Prince: The Vampire Prince, análisis...",NaN,El amor es uno de los temas universales que pe...,https://as.com/meristation/2020/09/21/analisis...
3,"Spinch, análisis PC y Switch: Una experiencia ...",NaN,No hay duda de que el género de las plataforma...,https://as.com/meristation/2020/09/20/analisis...
4,"Spelunky 2, análisis: el videojuego más hardco...",NaN,Mucho ha llovido desde el lanzamiento de Spelu...,https://as.com/meristation/2020/09/19/analisis...


In [34]:
#meristation.to_csv('../data/meristation_data.csv', index=False)

## Page parsing and link retrieving function for Meristation

In [7]:
def meristation_link_retrieve(num_pages):
    links = []
    
    # pages parser
    for i in range(num_pages):
        url = f'https://as.com/meristation/analisis/{331-i}'
        print(url)

        # building soup    
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'lxml')
        articles = soup.find_all('h2')

        # links retrieve
        for a in articles:
            if a.find('a') == None:
                pass
            else:
                links.append(a.find('a')['href'])
        
        print(links)
    
    #Deleting non review links
    for link in links:
        if 'analisis' not in link:
            links.remove(link)
                
    return links


In [8]:
num_pages = 3

links = meristation_link_retrieve(num_pages)

print(links)

https://as.com/meristation/analisis/331
['https://as.com/meristation/2020/10/03/analisis/1601701228_240717.html', 'https://as.com/meristation/2020/10/01/analisis/1601557244_384472.html', 'https://as.com/meristation/2020/09/30/analisis/1601454855_979194.html', 'https://as.com/meristation/2020/09/29/mexico/1601416248_335912.html', 'https://as.com/meristation/2020/09/29/analisis/1601370434_254241.html', 'https://as.com/meristation/2020/09/29/analisis/1601367548_437006.html', 'https://as.com/meristation/2020/09/28/analisis/1601283612_640735.html', 'https://as.com/meristation/2020/09/26/analisis/1601118163_185827.html', 'https://as.com/meristation/2020/09/26/analisis/1601109660_140327.html', 'https://as.com/meristation/2020/09/25/analisis/1601038731_079093.html', 'https://as.com/meristation/2020/09/25/analisis/1601019685_445862.html', 'https://as.com/meristation/2020/09/24/analisis/1600959846_926566.html', 'https://as.com/meristation/2020/09/23/analisis/1600848195_838475.html', 'https://as.